In [2]:
import pandas as pd
import fiona
import geopandas as gpd
%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns

import json
import requests
from tqdm import tqdm_notebook
from shapely.geometry import Polygon, MultiPolygon, shape

__Проверяю директорию, в которой сохранен данный ноутбук__

In [3]:
!pwd

/Users/juliatuvaleva/Documents/Python_test_new/LazyHackathon


__С помощью open загружаю json файл__

In [4]:
with open('../data-24713-2020-05-06.json', 'r', encoding='cp1251', errors='ignore') as data_file:
    data_loaded = json.load(data_file)

__Конвертирую загруженный json-файл в pandas dataframe__

In [5]:
df_pd = pd.DataFrame(data_loaded)

df_pd.head()

,AdmArea,AreaCoordinates,CadastralNumber,ChildEntries,ConsrtuctionWorkType,Coordinates,DeveloperInfo,DevelopmentArea,District,EntryAddReason,...,ID,MainFunctional,ObjectAddress,ObjectName,ObjectStatus,ParentEntries,Permission,SourceOfFinance,geoData,global_id
0,[Восточный административный округ],"37.693007979961,55.797586603633; 37.6930452283...",77:03:0003010:1008,[],новое строительство,"37.730303630757,55.776713288848",[{'OrgForm': 'Гаражно-строительный кооператив'...,1140.0,"[район Соколиная Гора, район Сокольники]",новый объект,...,776,Подземная стоянка,"Москва, ВАО, район Сокольники, улица Короленко...",Подземная автостоянка,в строительстве,[],[{'PermissionFile': [{'PermissionFile': 'fa383...,внебюджетные источники,"{'type': 'GeometryCollection', 'geometries': [...",272715485
1,"[Юго-Западный административный округ, Западный...","37.489585311279,55.6255477403393; 37.489012978...",77:06:0007002:40,[],новое строительство,"37.425808881626,55.662227471026",[{'OrgForm': 'Религиозная организация (объедин...,1295.0,"[район Тёплый Стан, район Солнцево]",новый объект,...,777,Храм,"Москва, ЮЗАО, Теплый Стан, Теплый стан ул. вл....",Храмовый комплекс,в строительстве,[],[{'PermissionFile': [{'PermissionFile': '1b7ad...,внебюджетные источники,"{'type': 'GeometryCollection', 'geometries': [...",272715489
2,"[Юго-Западный административный округ, Западный...","37.5690831463999,55.6853873827864; 37.56933548...",77:06:0003003:76,[],новое строительство,"37.530500567352,55.707343486183","[{'OrgForm': 'Акционерное общество', 'OrgName'...",14061.5,"[Академический район, район Раменки]",новый объект,...,779,Офисное здание,"Москва, ЮЗАО, район Академический, Профсоюзная...",Общественно-культурный центр с подземным гаражом,в строительстве,[],[{'PermissionFile': [{'PermissionFile': '76434...,внебюджетные источники,"{'type': 'GeometryCollection', 'geometries': [...",272715492
3,[Юго-Восточный административный округ],"37.7413731119759,55.690594894297; 37.741811903...",77:04:0004008:63,[],новое строительство,"37.741970758303,55.690432838423",[{'OrgName': 'МГПСС (ГКУ)'}],286.0,"[район Люблино, район Текстильщики]",новый объект,...,781,"Объект аварийно-спасательной службы, формирования","г. Москва, ЮВАО, район Текстильщики, улица Шку...",Поисково-спасательная станция (1-я очередь),в строительстве,[],[{'PermissionFile': [{'PermissionFile': 'e50c3...,городской бюджет,"{'type': 'GeometryCollection', 'geometries': [...",272715504
4,"[Центральный административный округ, Юго-Восто...","37.5947376978846,55.7714497629348; 37.59631350...",77:01:0004011:83,[],новое строительство,"37.682514557164,55.721826241818","[{'OrgForm': 'Закрытое акционерное общество', ...",4150.0,"[Тверской район, Южнопортовый район]",новый объект,...,751,Гостиница,"Москва, ЦАО, район Тверской, ул. 1-я Тверская-...",Гостиничный комплекс,в строительстве,[],[{'PermissionFile': [{'PermissionFile': 'a34d3...,внебюджетные источники,"{'type': 'GeometryCollection', 'geometries': [...",272715510


__Проверяю число строк в датафрэйме__

In [6]:
len(df_pd)

1572

__Выгружаю геометрию из джейсона__

In [8]:
df_pd = df_pd.dropna(subset = ['geoData'], axis = 0)

df_pd['geometry'] = df_pd['geoData'].apply(lambda x: shape(x))

## проверяю строки с пустой геометрией, если len == 0 строк, 
##значит геометрия полностью и корректно преобразована
len(df_pd[df_pd['geometry'].isnull()]) == 0

True

In [9]:
df_pd = df_pd.dropna(subset = ['geoData'], axis = 0)

len(df_pd)

1572

__Создаю геодатафрэйм__

In [10]:
df_pd = gpd.GeoDataFrame(data=df_pd, 
                               geometry = df_pd['geometry'], 
                               crs = {'init':'epsg:4326'})

df_pd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1572 entries, 0 to 1571
Data columns (total 27 columns):
AdmArea                 1572 non-null object
AreaCoordinates         1572 non-null object
CadastralNumber         1558 non-null object
ChildEntries            1572 non-null object
ConsrtuctionWorkType    1572 non-null object
Coordinates             1572 non-null object
DeveloperInfo           1572 non-null object
DevelopmentArea         1548 non-null float64
District                1572 non-null object
EntryAddReason          1572 non-null object
EntryState              1572 non-null object
FloorsAmount            1522 non-null object
FunctionalForNG         1572 non-null object
GPZUDocumentDate        1164 non-null object
GPZUDocumentNumber      1508 non-null object
GPZUDocumentStatus      1508 non-null object
ID                      1572 non-null int64
MainFunctional          1572 non-null object
ObjectAddress           1572 non-null object
ObjectName              1572 n

__Ивлечение необходимого типа геометрии из GeometryCollection__ <br> По сути это просто лист из двух и более типов объектов и нужно вытащить интересующий тип геометрии

In [13]:
def get_plgn(geom, shape_type):
    '''
    geom - колонка с геометрией объекта
    shape_type - тип геометрии, который нужно извлечь (POLYGON, POINT etc)
    '''
    geom_type = list(geom)
    target = [i for i in geom_type if shape_type in str(i) ]
    return target

In [14]:
df_pd['geometry'] = df_pd['geometry'].apply(lambda x: get_plgn(x, 'POLYGON')[0])

df_pd.columns

Index(['AdmArea', 'AreaCoordinates', 'CadastralNumber', 'ChildEntries',
       'ConsrtuctionWorkType', 'Coordinates', 'DeveloperInfo',
       'DevelopmentArea', 'District', 'EntryAddReason', 'EntryState',
       'FloorsAmount', 'FunctionalForNG', 'GPZUDocumentDate',
       'GPZUDocumentNumber', 'GPZUDocumentStatus', 'ID', 'MainFunctional',
       'ObjectAddress', 'ObjectName', 'ObjectStatus', 'ParentEntries',
       'Permission', 'SourceOfFinance', 'geoData', 'global_id', 'geometry'],
      dtype='object')

In [15]:
df_pd['AdmArea'] = df_pd['AdmArea'].apply(lambda x: x[0])
df_pd['DeveloperInfo'] = df_pd['DeveloperInfo'].apply(lambda x: x[0]['OrgName'])

In [17]:
cols = ['global_id', 'CadastralNumber', 'ObjectStatus', 'AdmArea','ConsrtuctionWorkType','DeveloperInfo','geometry']

df_pd[cols].head()

,global_id,CadastralNumber,ObjectStatus,AdmArea,ConsrtuctionWorkType,DeveloperInfo,geometry
0,272715485,77:03:0003010:1008,в строительстве,Восточный административный округ,новое строительство,Причал,"POLYGON ((37.69301 55.79759, 37.69305 55.79761..."
1,272715489,77:06:0007002:40,в строительстве,Юго-Западный административный округ,новое строительство,Местная религиозная организация православный п...,"POLYGON ((37.48959 55.62555, 37.48901 55.62576..."
2,272715492,77:06:0003003:76,в строительстве,Юго-Западный административный округ,новое строительство,СУ155,"POLYGON ((37.56908 55.68539, 37.56934 55.68540..."
3,272715504,77:04:0004008:63,в строительстве,Юго-Восточный административный округ,новое строительство,МГПСС (ГКУ),"POLYGON ((37.74137 55.69059, 37.74181 55.69040..."
4,272715510,77:01:0004011:83,в строительстве,Центральный административный округ,новое строительство,ФСК Русское ЗолотоСтройинвест,"POLYGON ((37.59474 55.77145, 37.59631 55.77053..."


__Экспорт файла__

In [18]:
df_pd[cols].to_file('../msc_construction_upd.geojson', 'GeoJSON')